In [1]:
import os
import numpy as np
import pandas as pd
import gc
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [2]:
dias = ["02-03-2018.csv", "21-02-2018.csv", "15-02-2018.csv", "16-02-2018.csv" ,"01-03-2018.csv", "28-02-2018.csv", "14-02-2018.csv", "22-02-2018.csv", "23-02-2018.csv"]

In [3]:
dfTotal = pd.DataFrame()
i = 0
for dia in dias:
    dfAux = pd.read_csv("../../Dataset/" + dia, low_memory = False)
    dfAux = dfAux.drop([0,1])
    dfTotal = pd.concat([dfTotal, dfAux])
    print("Ficheiro " + dia + " carregado")
    i += 1

Ficheiro 02-03-2018.csv carregado
Ficheiro 21-02-2018.csv carregado
Ficheiro 15-02-2018.csv carregado
Ficheiro 16-02-2018.csv carregado
Ficheiro 01-03-2018.csv carregado
Ficheiro 28-02-2018.csv carregado
Ficheiro 14-02-2018.csv carregado
Ficheiro 22-02-2018.csv carregado
Ficheiro 23-02-2018.csv carregado


In [4]:
dfTotal

,Dst Port,Fwd PSH Flags,Protocol,Bwd PSH Flags,Flow Duration,Fwd URG Flags,Tot Fwd Pkts,Bwd URG Flags,Tot Bwd Pkts,FIN Flag Cnt,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
2,443.0,0,6.0,0,141385.0,0,9.0,0,7.0,0,...,4.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,49684.0,1,6.0,0,281.0,0,2.0,0,1.0,0,...,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,443.0,0,6.0,0,279824.0,0,11.0,0,15.0,0,...,5.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,443.0,0,6.0,0,132.0,0,2.0,0,0.0,0,...,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,443.0,0,6.0,0,274016.0,0,9.0,0,13.0,0,...,5.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042864,54116.0,0,6.0,0,85992425.0,0,2.0,0,0.0,0,...,0.0,20.0,0.0,0.0,0.0,0.0,85992425.0,0.0,85992425.0,85992425.0
1042865,33914.0,0,6.0,0,132.0,0,2.0,0,0.0,0,...,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1042866,53.0,0,17.0,0,79134.0,0,1.0,0,1.0,0,...,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1042867,3389.0,0,6.0,0,10053785.0,0,11.0,0,12.0,0,...,7.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
input_labelAE = np.array(dfTotal.loc[:, dfTotal.columns != "Label"]).astype(np.float)

In [6]:
dfTotal = None
gc.collect()

52

In [7]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(input_labelAE)
input_labelAE = scaler.transform(input_labelAE)

In [8]:
input_labelAE = shuffle(input_labelAE)

In [9]:
inp_train,inp_test,out_train,out_test = train_test_split(input_labelAE, input_labelAE, test_size=0.2)

In [10]:
input_labelAE = None
gc.collect()

In [11]:
input_model = keras.layers.Input(shape = (78,))
enc = keras.layers.Dense(units = 64, activation = "relu", use_bias = True)(input_model)
enc = keras.layers.Dense(units = 36, activation = "relu", use_bias = True)(enc)
enc = keras.layers.Dense(units = 18, activation = "relu")(enc)
dec = keras.layers.Dense(units = 36, activation = "relu", use_bias = True)(enc)
dec = keras.layers.Dense(units = 64, activation = "relu", use_bias = True)(dec)
dec = keras.layers.Dense(units = 78, activation = "relu", use_bias = True)(dec)
auto_encoder = keras.Model(input_model, dec)

In [12]:
encoder = keras.Model(input_model, enc)
decoder_input = keras.layers.Input(shape = (18,))
decoder_layer = auto_encoder.layers[-3](decoder_input)
decoder_layer = auto_encoder.layers[-2](decoder_layer)
decoder_layer = auto_encoder.layers[-1](decoder_layer)
decoder = keras.Model(decoder_input, decoder_layer)

In [13]:
auto_encoder.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00025), loss = "mean_squared_error", metrics = ['accuracy'])

In [14]:
train = auto_encoder.fit(x = np.array(inp_train), y = np.array(out_train),validation_split= 0.1, epochs = 10, verbose = 1, shuffle = True)

Epoch 1/10
185578/185578 [==============================] - 222s 1ms/step - loss: 0.0186 - accuracy: 0.2971 - val_loss: 0.0177 - val_accuracy: 0.24456s - loss: 0.0187 - acc - ETA: 35s - loss: 0 - ETA: 34s - loss: 0.0187 - accuracy: - ETA: 34s - loss: 0.0187 - accuracy:  - ETA: 33s - loss: 0.0187 - accuracy: 0.2 - ETA: 33s - loss: 0.0187 - accuracy - ETA: 33s - loss: 0.0187 - accuracy - ETA: 32s - loss: 0.0187 - ac - ETA: 32s - loss: 0.0187 - accuracy: 0.2 - ETA: 
Epoch 2/10
185578/185578 [==============================] - 204s 1ms/step - loss: 0.0177 - accuracy: 0.3439 - val_loss: 0.0176 - val_accuracy: 0.5515
Epoch 3/10
185578/185578 [==============================] - 216s 1ms/step - loss: 0.0176 - accuracy: 0.3518 - val_loss: 0.0176 - val_accuracy: 0.1214
Epoch 4/10
185578/185578 [==============================] - 204s 1ms/step - loss: 0.0176 - accuracy: 0.3746 - val_loss: 0.0175 - val_accuracy: 0.3909
Epoch 5/10
185578/185578 [==============================] - 204s 1ms/step - loss: 

In [16]:
predict = auto_encoder.predict(np.array(inp_test))

In [17]:
losses = keras.losses.mean_squared_error(out_test, predict).numpy()

In [18]:
total = 0
for loss in losses:
    total += loss
print(total / len(losses))

0.015983125285722512


In [19]:
encoder.save("Encoder.h5")